In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

import asyncio

from codecs import StreamReader
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow

api_key = os.getenv("OPEN_ROUTER_API_KEY") 

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    model="openai/gpt-4o-mini",    
    model_info={
        "family": "openai",
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "structured_output": True
    },
)

# Sequential Flow

In [12]:
writer = AssistantAgent(
    name="Writer",
    description="A writer agent that generates text based on user input.",
    model_client=model_client,
    system_message="You are a creative writer. Please write a story based on the user's input.",
)

reviewer = AssistantAgent(
    name="Reviewer",
    description="A reviewer agent that provides feedback on the text generated by the writer.",
    model_client=model_client,
    system_message="You are a reviewer. Please provide feedback on the text generated by the writer.",
)

In [13]:
builder = DiGraphBuilder()
builder.add_node(writer).add_node(reviewer)
builder.add_edge(writer, reviewer)

graph = builder.build()

In [14]:
graph

DiGraph(nodes={'Writer': DiGraphNode(name='Writer', edges=[DiGraphEdge(target='Reviewer', condition=None, condition_function=None, activation_group='Reviewer', activation_condition='all')], activation='all'), 'Reviewer': DiGraphNode(name='Reviewer', edges=[], activation='all')}, default_start_node=None)

In [15]:
team = GraphFlow([writer,reviewer], graph)

In [16]:
stream = team.run_stream(task ="Write a good poem about India in less than 30 words.")

async for event in stream:
    print(event)

id='f0f3ddac-7e91-421a-9e8a-2caafae05f57' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 2, 15, 17, 23, 53, 138181, tzinfo=datetime.timezone.utc) content='Write a good poem about India in less than 30 words.' type='TextMessage'


id='b41008ff-7956-441c-b91f-32bcfb927003' source='Writer' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=41) metadata={} created_at=datetime.datetime(2026, 2, 15, 17, 23, 55, 88705, tzinfo=datetime.timezone.utc) content='Land of vibrant hues, where cultures entwine,  \nGanges whispers stories, ancient and divine.  \nMountains stand tall, with hearts so wide,  \nIn unity and pride, India abides.' type='TextMessage'
id='f8689019-d516-43d9-b22f-92ae748dbc80' source='Reviewer' models_usage=RequestUsage(prompt_tokens=88, completion_tokens=85) metadata={} created_at=datetime.datetime(2026, 2, 15, 17, 23, 57, 88292, tzinfo=datetime.timezone.utc) content='This poem beautifully captures the essence of India through vivid imagery and emotion. The use of "vibrant hues" effectively conveys the diversity of cultures. The imagery of the Ganges and mountains adds depth and connects nature with spirituality. The closing line emphasizes unity and pride, resonating with the overall theme.

# Parallel Fan out

In [17]:
agent_a = AssistantAgent("A", model_client=model_client, system_message="You are a helpful assistant.")
agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to Chinese.")
agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Japanese.")

# Create a directed graph with fan-out flow A -> (B, C).
builder = DiGraphBuilder()
builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
builder.add_edge(agent_a, agent_b).add_edge(agent_a, agent_c)
graph = builder.build()

# Create a GraphFlow team with the directed graph.
team = GraphFlow(
    participants=[agent_a, agent_b, agent_c],
    graph=graph,
    termination_condition=MaxMessageTermination(5),
)

# Run the team and print the events.
async for event in team.run_stream(task="Write a short story about a cat."):
    print(event)

id='2b1ff277-78ec-40c8-90bb-f0d3e52ba654' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 2, 15, 17, 23, 57, 126029, tzinfo=datetime.timezone.utc) content='Write a short story about a cat.' type='TextMessage'
id='e0ed2a7e-c1b1-4dd9-8aa5-b6a9ea13ed60' source='A' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=667) metadata={} created_at=datetime.datetime(2026, 2, 15, 17, 24, 9, 943452, tzinfo=datetime.timezone.utc) content='Once upon a time in a quaint little village, there lived a curious cat named Whiskers. Whiskers was not your ordinary feline; he was a fluffy orange tabby with bright green eyes that sparkled with mischief. Every day, he would venture out of his cozy home, a small cottage owned by a kind elderly woman named Mrs. Maple.\n\nOne sunny morning, Whiskers decided to explore beyond the familiar cobblestone streets of the village. He slinked through the narrow alleys and ventured into the vibrant meadow that bordered the village. 

# Cindtional Branching (A->B ("yes"), A->C (otherwise))

In [ ]:
agent_a = AssistantAgent(
        "A",
        model_client=model_client,
        system_message="Detect if the input is in Chinese. If it is, say 'yes', else say 'no', and nothing else.",
    )
agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to English.")
agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Chinese.")

# Create a directed graph with conditional branching flow A -> B ("yes"), A -> C (otherwise).
builder = DiGraphBuilder()
builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
# Create conditions as callables that check the message content.
builder.add_edge(agent_a, agent_b, condition=lambda msg: "yes" in msg.to_model_text())
builder.add_edge(agent_a, agent_c, condition=lambda msg: "yes" not in msg.to_model_text())
graph = builder.build()

# Create a GraphFlow team with the directed graph.
team = GraphFlow(
    participants=[agent_a, agent_b, agent_c],
    graph=graph,
    termination_condition=MaxMessageTermination(5),
)

# Run the team and print the events.
async for event in team.run_stream(task="AutoGen is a framework for building AI agents."):
    print(event)

id='a9fe2a60-4f05-489c-ba14-90544ca2d64e' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 2, 15, 17, 26, 40, 138258, tzinfo=datetime.timezone.utc) content='AutoGen is a framework for building AI agents.' type='TextMessage'
id='df4ffc72-b84a-4fe6-b2c9-7f3ed96554c8' source='A' models_usage=RequestUsage(prompt_tokens=47, completion_tokens=1) metadata={} created_at=datetime.datetime(2026, 2, 15, 17, 26, 41, 448551, tzinfo=datetime.timezone.utc) content='no' type='TextMessage'
id='abc41be8-ecf9-474a-8a4e-a9f7724f2c8b' source='C' models_usage=RequestUsage(prompt_tokens=33, completion_tokens=12) metadata={} created_at=datetime.datetime(2026, 2, 15, 17, 26, 42, 155815, tzinfo=datetime.timezone.utc) content='AutoGen是一个构建AI代理的框架。' type='TextMessage'
messages=[TextMessage(id='a9fe2a60-4f05-489c-ba14-90544ca2d64e', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 15, 17, 26, 40, 138258, tzinfo=datetime.timezone.utc), content='AutoG